In [ ]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [161]:
# Load anonymized data
import os

data_path = "Anon_Data"
fuel_df = pd.read_excel(os.path.join(data_path, "fuel_data_anon.xlsx"))
vehicle_df = pd.read_excel(os.path.join(data_path, "vehicle_data_anon.xlsx"))
mileage_df = pd.read_excel(os.path.join(data_path, "mileage_data_anon.xlsx"))


In [ ]:
#Cleaning registration numbers
fuel_df = fuel_df.drop(["CardNumber", "Driver Name", "Branch Name"], axis = 1)
fuel_df = fuel_df.rename(columns={"RegistrationNumber": "registration"})
fuel_df['registration'] = fuel_df['registration'].astype(str).str.strip()
fuel_df['registration'].replace('-', pd.NA, inplace=True)
fuel_df = fuel_df.dropna(subset=['registration'])

In [ ]:
#Filtering to only valid fuel types as others are negligible in number
valid_fuels = [
    'Unleaded - Medium octane',
    'Diesel',
    'V Power Diesel',
    'Unleaded High Perf'    
]

fuel_df = fuel_df[fuel_df['ProductName'].isin(valid_fuels)]
fuel_df['ProductName'].value_counts()

In [ ]:
#Converting TransactionDate to year and month columns
fuel_df['date'] = pd.to_datetime(fuel_df['FuelMonthName'], format='%b %Y', errors='coerce')

fuel_df['year'] = fuel_df['date'].dt.year
fuel_df['month'] = fuel_df['date'].dt.month   # 1–12 (SAFE)

fuel_df = fuel_df.drop(["TransactionDate", "date", "FuelMonthName"], axis=1)

In [ ]:
#Creating active vehicle dataframe for testing
vehicle_df['is_active'] = vehicle_df['Vehicle Status'].isin(['Current - On Road', 'Current - Off Road'])
active_vehicles = vehicle_df[vehicle_df['is_active']]

In [ ]:
#Only using vans as cars have limited trackers
valid_vehicles = [
    'Small Van',
    'Medium Van',
    'Large Van'    
]

vehicle_df = vehicle_df[vehicle_df['Asset Type'].isin(valid_vehicles)]
vehicle_df['Asset Type'].value_counts()

In [ ]:
#Dropping unneeded columns
vehicle_df = vehicle_df.drop("Derivative", axis = 1)

In [ ]:
#Loading and preparing mileage data
mileage_df = mileage_df.rename(columns={'Year': 'year'})
mileage_df = mileage_df.rename(columns={'Month': 'month'})

In [ ]:
#Removing invalid month entries
mileage_df = mileage_df[mileage_df['month'] != '-']

#Converting month names to month numbers
mileage_df['month'] = pd.to_datetime(
    mileage_df['month'],
    format='%b',
    errors='coerce'
).dt.month

In [ ]:
# 1️⃣ Strip column names first (safest with Excel files)
fuel_df.columns = fuel_df.columns.str.strip()
mileage_df.columns = mileage_df.columns.str.strip()
vehicle_df.columns = vehicle_df.columns.str.strip()

# 2️⃣ Rename registration to RegistrationNumber
fuel_df = fuel_df.rename(columns={'registration': 'RegistrationNumber'})

# 3️⃣ Standardize RegistrationNumber as string in all dataframes
fuel_df['RegistrationNumber'] = fuel_df['RegistrationNumber'].astype(str).str.strip()
mileage_df['RegistrationNumber'] = mileage_df['RegistrationNumber'].astype(str).str.strip()
vehicle_df['RegistrationNumber'] = vehicle_df['RegistrationNumber'].astype(str).str.strip()

# 4️⃣ Ensure year/month numeric for merge
fuel_df['year'] = pd.to_numeric(fuel_df['year'], errors='coerce').astype('Int64')
fuel_df['month'] = pd.to_numeric(fuel_df['month'], errors='coerce').astype('Int64')

mileage_df['year'] = pd.to_numeric(mileage_df['year'], errors='coerce').astype('Int64')
mileage_df['month'] = pd.to_numeric(mileage_df['month'], errors='coerce').astype('Int64')

# 5️⃣ Merge fuel with mileage
fuel_mileage_df = fuel_df.merge(
    mileage_df,
    on=['RegistrationNumber', 'year', 'month'],
    how='left'
)

# 6️⃣ Merge with vehicle data
full_df = fuel_mileage_df.merge(
    vehicle_df,
    on='RegistrationNumber',
    how='left'
)

# 7️⃣ Drop duplicate columns if any
full_df = full_df.loc[:, ~full_df.columns.duplicated()]

# 8️⃣ Filter to only data from April 2023 onwards
full_df = full_df[
    (full_df['year'] > 2023) |
    ((full_df['year'] == 2023) & (full_df['month'] >= 4))
]


# 8️⃣ Filter to only valid registrations present in all three dataframes
valid_regs = (
    set(fuel_df['RegistrationNumber'])
    & set(vehicle_df['RegistrationNumber'])
    & set(mileage_df['RegistrationNumber'])
)

full_df = full_df[full_df['RegistrationNumber'].isin(valid_regs)]



In [ ]:
# --- 1️⃣ Drop rows missing critical vehicle info ---
full_df = full_df.dropna(subset=['Make','Model'])

# --- 2️⃣ Handle mileage data ---
# Create a flag for vehicles with trackers
full_df['has_tracker'] = full_df['Total Miles'].notna()

# --- 3️⃣ Final check ---
print(full_df[['RegistrationNumber','Make','Model','has_tracker','Average Miles Per Day','Total Miles']].head(10))
print(full_df.shape)
print(full_df['has_tracker'].value_counts())

In [ ]:
#Creating Lag features to capture previous months
full_df = full_df.sort_values(['RegistrationNumber','year','month'])
full_df['Fuel_Lag1'] = full_df.groupby('RegistrationNumber')['Net Amount'].shift(1)
full_df['Fuel_Lag2'] = full_df.groupby('RegistrationNumber')['Net Amount'].shift(2)
full_df['Fuel_Lag6'] = full_df.groupby('RegistrationNumber')['Net Amount'].shift(6)

In [ ]:
#Create a 3 months rolling average to help with prediction
full_df['Fuel_3mo_avg'] = full_df.groupby('RegistrationNumber')['Net Amount'].transform(
    lambda x: x.shift(1).rolling(window=3, min_periods=1).mean()
)
#6-month rolling average (excluding current month)
full_df['Fuel_6mo_avg'] = full_df.groupby('RegistrationNumber')['Net Amount'].shift(1).rolling(window=6, min_periods=1).mean()

In [ ]:
full_df = full_df.sort_values(['RegistrationNumber', 'year', 'month'])

full_df['Mileage_Lag1'] = (
    full_df.groupby('RegistrationNumber')['Total Miles'].shift(1)
)

full_df['Mileage_3mo_avg'] = (
    full_df.groupby('RegistrationNumber')['Total Miles']
    .shift(1)
    .rolling(3)
    .mean()
)

In [ ]:
#Vreating more variables to try improve model
full_df['Fuel_per_mile'] = full_df['Net Amount'] / full_df['Total Miles']
full_df['Average Miles Per Day'] = pd.to_numeric(
    full_df['Average Miles Per Day'],
    errors='coerce'
)

full_df['Registered Date'] = pd.to_datetime(
    full_df['Registered Date'],
    errors='coerce'
)

full_df['Vehicle_Age'] = full_df['year'] - full_df['Registered Date'].dt.year
full_df['Vehicle_Age'] = pd.to_numeric(
    full_df['Vehicle_Age'],
    errors='coerce'
)

In [ ]:
#Preparing data for modeling
y = full_df['Net Amount']
X = full_df[[
    'Fuel_Lag1', 'Fuel_Lag2', 'Fuel_Lag6',         # lag features
    'Fuel_3mo_avg', 'Fuel_6mo_avg',              # rolling averages
    'Mileage_3mo_avg', 'Mileage_Lag1', 'Fuel_per_mile', # mileage
    'has_tracker',                      # missing mileage flag
    'Make', 'Model', 'ProductName', 'Branch Name', 'Vehicle_Age'  # categorical
]]

#Splitting data into training and testing sets based on date
full_df['date'] = pd.to_datetime(
    full_df['year'].astype(str) + '-' +
    full_df['month'].astype(str) + '-01'
)

train_end = pd.Timestamp('2025-09-30')

#Creating masks for training and testing sets
train_mask = full_df['date'] <= train_end
test_mask  = full_df['date'] > train_end

#Splitting data into training and testing sets based on date
X_train = X.loc[train_mask].copy()
y_train = y.loc[train_mask].copy()

X_test  = X.loc[test_mask].copy()
y_test  = y.loc[test_mask].copy()




In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

categorical_cols = ['Make', 'Model', 'ProductName', 'Branch Name']

# Ensure categorical columns are properly typed
for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

# Define the model
model = lgb.LGBMRegressor(
    objective='regression',
    learning_rate=0.05,
    num_leaves=31,
    n_estimators=500
)

# Fit the model WITHOUT verbose or early stopping
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse'
)

# Predict
y_pred = model.predict(X_test)

# Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R2: {r2:.2f}")

In [ ]:
#Feature Importance
fi = pd.Series(
    model.feature_importances_,
    index=X_train.columns
).sort_values(ascending=False)

fi.head(10)

In [ ]:
#Plotting Actual vs Predicted
import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred, alpha=0.3)
plt.xlabel("Actual Fuel Spend")
plt.ylabel("Predicted Fuel Spend")
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()])
plt.show()

In [ ]:
#Plotting Residuals
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.3)
plt.axhline(0)
plt.show()

In [ ]:
# Keep only Jan 2026 records
latest_df = full_df[
    (full_df['year'] == 2026) & (full_df['month'] == 1)
].copy()

# Keep only one row per vehicle (latest in case there are multiple entries in Jan)
latest_df = (
    latest_df.sort_values(['RegistrationNumber', 'year', 'month'])
    .groupby('RegistrationNumber')
    .tail(1)
    .copy()
)

forecast_df = latest_df[['RegistrationNumber', 'Branch Name']].copy()
recursive_df = latest_df.copy()

from calendar import month_name

pred_columns = []

for i in range(1, 4):
    # Update month/year
    recursive_df['month'] += 1
    recursive_df['year'] += (recursive_df['month'] - 1) // 12
    recursive_df['month'] = ((recursive_df['month'] - 1) % 12) + 1
    
    # Dynamic column name like Pred_Feb2026
    col_name = f"Pred_{month_name[recursive_df['month'].iloc[0]][:3]}{recursive_df['year'].iloc[0]}"
    pred_columns.append(col_name)
    
    # Prepare features
    future_X = recursive_df[X_train.columns].copy()
    categorical_cols = ['Make', 'Model', 'ProductName', 'Branch Name']
    for col in categorical_cols:
        future_X[col] = future_X[col].astype('category')
        future_X[col] = future_X[col].cat.set_categories(X_train[col].cat.categories)
    
    # Predict
    pred = model.predict(future_X)
    forecast_df[col_name] = pred
    
    # Update lag features for next month
    recursive_df['Fuel_Lag6'] = recursive_df['Fuel_Lag2']
    recursive_df['Fuel_Lag2'] = recursive_df['Fuel_Lag1']
    recursive_df['Fuel_Lag1'] = pred
    
    # Update rolling averages
    recursive_df['Fuel_3mo_avg'] = recursive_df[['Fuel_Lag1','Fuel_Lag2','Fuel_Lag6']].mean(axis=1)
    recursive_df['Fuel_6mo_avg'] = recursive_df[['Fuel_Lag1','Fuel_Lag2','Fuel_Lag6']].mean(axis=1)

forecast_df.head(10)
